<a href="https://colab.research.google.com/github/ArtemBaron/baron_vergeles/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

In [2]:
import pandas as pd
import numpy as np

In [3]:
def select(sql):
  return pd.read_sql(sql, con)

In [4]:
! ls '/content/drive/My Drive/data'

db  german_credit.db


In [5]:
con = sqlite3.connect('db')

In [6]:
sql = '''
drop table if exists client;
create table client
(   
    client_id char(10),  -- ID клиента
    full_name varchar(100), -- ФИО клиента
    birth_dt date -- дата рождения
);
insert into client values ('PCLTN12345', 'Барабушка Пётр Сергеевич', '1970-01-01');
insert into client values ('PBVNT67890', 'Ульянов Владимир Ильич', '1987-04-03');
insert into client values ('ACTGH12345', 'Грызько Анастасия Викторовна', '1988-07-28');
create table transactions
(
    transaction_id int,  -- ID транзакции
    transaction_dt date, -- дата и время транзакции (с 01.01.2020 по 31.12.2020)
    amount_rur decimal(10,2),  -- сумма транзакции
    success_flg char(1), -- флаг успешности транзакции - Y/N
    client_id char(10)  -- ID клиента
);
insert into transactions values (1, '2020-01-01', 20.51, 'Y', 'PCLTN12345');
insert into transactions values (2, '2020-01-02', 1570.22, 'Y', 'PCLTN12345');
insert into transactions values (3, '2020-02-03', 104.32, 'N', 'PCLTN12345');
insert into transactions values (4, '2020-02-03', 104.32, 'Y', 'PCLTN12345');
insert into transactions values (5, '2020-02-05', 25000.00, 'Y', 'PCLTN12345');
insert into transactions values (6, '2020-04-04', 17878.02, 'Y', 'PCLTN12345');

insert into transactions values (7, '2020-04-12', 89.51, 'Y', 'PBVNT67890');
insert into transactions values (8, '2020-04-11', 70.22, 'Y', 'PBVNT67890');
insert into transactions values (9, '2020-04-08', 7.32, 'N', 'PBVNT67890');
insert into transactions values (10, '2020-04-09', 10.32, 'Y', 'PBVNT67890');
insert into transactions values (11, '2020-04-10', 4.02, 'Y', 'PBVNT67890');
insert into transactions values (12, '2020-04-01', 15.20, 'Y', 'PBVNT67890');

insert into transactions values (13, '2020-02-05', 100000.00, 'Y', 'ACTGH12345');
insert into transactions values (14, '2020-03-05', 100000.00, 'Y', 'ACTGH12345');
insert into transactions values (15, '2020-04-05', 100000.00, 'Y', 'ACTGH12345');
insert into transactions values (16, '2020-05-05', 200000.00, 'Y', 'ACTGH12345');
insert into transactions values (17, '2020-06-05', 150000.00, 'Y', 'ACTGH12345');
insert into transactions values (18, '2020-07-05', 120000.00, 'Y', 'ACTGH12345');
'''

In [7]:
cur = con.cursor()

In [8]:
cur.executescript(sql)

In [9]:
sql = '''
select *
from transactions t
'''

In [10]:
select(sql)

,transaction_id,transaction_dt,amount_rur,success_flg,client_id
0,1,2020-01-01,20.51,Y,PCLTN12345
1,2,2020-01-02,1570.22,Y,PCLTN12345
2,3,2020-02-03,104.32,N,PCLTN12345
3,4,2020-02-03,104.32,Y,PCLTN12345
4,5,2020-02-05,25000.00,Y,PCLTN12345
5,6,2020-04-04,17878.02,Y,PCLTN12345
6,7,2020-04-12,89.51,Y,PBVNT67890
7,8,2020-04-11,70.22,Y,PBVNT67890
8,9,2020-04-08,7.32,N,PBVNT67890
9,10,2020-04-09,10.32,Y,PBVNT67890


In [11]:
sql = '''
select t.client_id, c.full_name, t.amount_rur
from transactions t
left join client c on t.client_id = c.client_id
where date(t.transaction_dt, 'start of month') = '2020-04-01' and t.success_flg = 'Y' and t.amount_rur > 1000
group by t.client_id
'''

In [12]:
select(sql)

,client_id,full_name,amount_rur
0,ACTGH12345,Грызько Анастасия Викторовна,100000.00
1,PCLTN12345,Барабушка Пётр Сергеевич,17878.02


In [23]:
sql = '''
with ranking as (select *, row_number() over (partition by t.client_id order by t.amount_rur desc) as rank
from transactions t
where t.success_flg = 'Y')

select t.client_id, t.transaction_id, t.amount_rur
from ranking t
where t.rank = 1 or t.rank = 2
'''

In [24]:
select(sql)

,client_id,transaction_id,amount_rur
0,ACTGH12345,16,200000.00
1,ACTGH12345,17,150000.00
2,PBVNT67890,7,89.51
3,PBVNT67890,8,70.22
4,PCLTN12345,5,25000.00
5,PCLTN12345,6,17878.02


In [42]:
sql = '''
with test as (select *, lead(t.amount_rur) over (partition by t.client_id order by t.transaction_dt) as next_amount,
lag(t.amount_rur) over (partition by t.client_id order by t.transaction_dt) as prev_amount
from transactions t
where t.success_flg = 'Y'
)
select t.client_id, t.transaction_id, t.next_amount, t.prev_amount
from test t
where t.amount_rur > t.next_amount and t.amount_rur > t.prev_amount
'''

In [43]:
select(sql)

,client_id,transaction_id,next_amount,prev_amount
0,ACTGH12345,16,150000.00,100000.00
1,PCLTN12345,2,104.32,20.51
2,PCLTN12345,5,17878.02,104.32


In [52]:
sql = '''
select t.transaction_dt, t.amount_rur, sum(amount_rur) over (order by t.transaction_dt asc) as cum_amount_rur
from transactions t
where t.success_flg = 'Y'
'''

In [53]:
select(sql)

,transaction_dt,amount_rur,cum_amount_rur
0,2020-01-01,20.51,20.51
1,2020-01-02,1570.22,1590.73
2,2020-02-03,104.32,1695.05
3,2020-02-05,25000.00,126695.05
4,2020-02-05,100000.00,126695.05
5,2020-03-05,100000.00,226695.05
6,2020-04-01,15.20,226710.25
7,2020-04-04,17878.02,244588.27
8,2020-04-05,100000.00,344588.27
9,2020-04-09,10.32,344598.59


In [86]:
cur = con.cursor()

In [87]:
cur.executescript('drop table if exists copied;')

In [89]:
cur.executescript('CREATE TABLE copied AS SELECT * FROM transactions WHERE 0;')

In [91]:
sql = '''
select * from copied
'''

In [92]:
select(sql)

,transaction_id,transaction_dt,amount_rur,success_flg,client_id
